# Simulating sets of data using RocketPy

In [1]:
!pip install rocketpy
!curl -o NACA0012-radians.txt https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/airfoils/NACA0012-radians.txt
!curl -o Cesaroni_M1670.eng https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/motors/cesaroni/Cesaroni_M1670.eng
!curl -o powerOffDragCurve.csv https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/rockets/calisto/powerOffDragCurve.csv
!curl -o powerOnDragCurve.csv https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/rockets/calisto/powerOnDragCurve.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1052  100  1052    0     0  12991      0 --:--:-- --:--:-- --:--:-- 13150
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   2536      0 --:--:-- --:--:-- --:--:--  2546
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3362  100  3362    0     0  37331      0 --:--:-- --:--:-- --:--:-- 37355
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3362  100  3362    0     0  46584      0 --:--:-- --:--:-- --:--:-- 46694


### Setting up the simulation

In [2]:
from rocketpy import Environment, Flight, Rocket, Fluid, CylindricalTank, MassFlowRateBasedTank, HybridMotor
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline


############################################################################
#Engine Settings 

# Define the fluids
oxidizer_liq = Fluid(name="N2O_l", density=1220)
oxidizer_gas = Fluid(name="N2O_g", density=1.9277)

# Define tank geometry
tank_shape = CylindricalTank(115 / 2000, 0.705)

# Define tank
oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=tank_shape,
    flux_time=5.2,
    initial_liquid_mass=4.11,
    initial_gas_mass=0,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=(4.11 - 0.5) / 5.2,
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)

example_hybrid = HybridMotor(
    thrust_source=lambda t: 2000 - (2000 - 1400) / 5.2 * t,
    dry_mass=2,
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=63.36 / 2000,
    grain_number=4,
    grain_separation=0,
    grain_outer_radius=0.0575,
    grain_initial_inner_radius=0.025,
    grain_initial_height=0.1375,
    grain_density=900,
    grains_center_of_mass_position=0.384,
    center_of_dry_mass_position=0.284,
    nozzle_position=0,
    burn_time=5.2,
    throat_radius=26 / 2000,
)

example_hybrid.add_tank(
  tank = oxidizer_tank, position = 1.0615
)


############################################################################
#Rocket Configuration 

calisto = Rocket(
    radius=127 / 2000,
    mass=14.426,
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="powerOffDragCurve.csv",
    power_on_drag="powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

rail_buttons = calisto.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.618,
    angular_position=45,
)
nose_cone = calisto.add_nose(length=0.55829, kind="vonKarman", position=1.278)
fin_set = calisto.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.060,
    span=0.110,
    position=-1.04956,
    cant_angle=0.5,
    airfoil=("NACA0012-radians.txt", "radians"),
)
tail = calisto.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)
main = calisto.add_parachute(
    "Main",
    cd_s=10.0,
    trigger=800,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)
drogue = calisto.add_parachute(
    "Drogue",
    cd_s=1.0,
    trigger="apogee",
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

calisto.add_motor(example_hybrid, position=-1.255)

In [5]:
# import datetime
# ############################################################################
# #Environment Settings 
# env = Environment(latitude=35.34723084964506, longitude=-117.81006429132387, elevation=624.14)

# #three potential launch dates
# today = datetime.datetime(2026, 2, 9)
# # day1 = datetime.datetime(2026, 5, 30)
# # day2 = datetime.datetime(2026, 5, 31)
# # day3 = datetime.datetime(2026, 6, 1)

# env.set_date(
#     (today.year, today.month, today.day, 12)
# )  # Hour given in UTC time
# env.set_atmospheric_model(type="Forecast", file="GFS") 

In [6]:
# from rocketpy.simulation.flight_data_exporter import FlightDataExporter

# test_flight = Flight(
#     rocket=calisto, environment=env, rail_length=5.2, inclination=85, heading=0, max_time_step=0.01,
# )

# # Create an exporter object
# exporter = FlightDataExporter(test_flight)

# exporter.export_data(
#     "calisto_flight_data.csv",
#     "acceleration",
#     "vz",
#     "z",
# )

# print("Flight data exported to fcalisto_light_data.csv")

### This is where I will start generating the larger set of flights

In [12]:
import datetime
import numpy as np
from rocketpy.simulation.flight_data_exporter import FlightDataExporter

nominal_mass = 14.426 #placeholder
nominal_cg = 0 #placeholder 

N = 2 #runs per potential day
month_day = [["28", "05"], ["29", "05"], ["30", "05"], ["31", "05"], ["01", "06"], ["02", "06"]]
years = ["2023", "2024", "2025"]
#This gives us 18 different days to work with 


runs = 0
for i in range(len(years)):
    for j in range(len(month_day)):
        #this is sounding data from Las Vegas via University of Wyoming 
        url = f"https://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR={years[i]}&MONTH=0{month_day[j][1]}&FROM={month_day[j][0]}12&TO=3012&STNM=72388"
        env = Environment()
        env.set_atmospheric_model(type="wyoming_sounding", file=url)

        for k in range(N):
            calisto.mass = np.random.normal(nominal_mass, 0.1 * nominal_mass)
            calisto.center_of_mass = nominal_cg + np.random.normal(0, 0.05)

            flight = Flight(
                rocket=calisto,
                environment=env,
                rail_length=15.24, #FAR rail 
                inclination= np.random.normal(90, 5), #degrees, gaussian noise with mean 90, SD 5
                heading= np.random.normal(0, 4), #degrees, gaussian noise mean 0, SD 4
                max_time_step=0.01, #needed for good logging 
            )
            
            runs += 1
            exporter = FlightDataExporter(flight)
            exporter.export_data(
                f"flights/wyoming_data_{runs}.csv",
                "acceleration",
                "vz",
                "z",
            )
            print(f"Run {runs}: apogee = {flight.apogee:.1f} m")

Run 1: apogee = 5031.1 m
Run 2: apogee = 5071.5 m
Run 3: apogee = 4777.9 m
Run 4: apogee = 5073.8 m
Run 5: apogee = 5170.1 m
Run 6: apogee = 5141.2 m
Run 7: apogee = 5166.5 m
Run 8: apogee = 4970.4 m
Run 9: apogee = 5176.4 m
Run 10: apogee = 5145.7 m
Run 11: apogee = 5161.4 m
Run 12: apogee = 5166.4 m
Run 13: apogee = 5197.7 m
Run 14: apogee = 5032.6 m
Run 15: apogee = 4935.6 m
Run 16: apogee = 5044.5 m
Run 17: apogee = 5126.6 m
Run 18: apogee = 5049.6 m
Run 19: apogee = 5217.9 m
Run 20: apogee = 5199.0 m
Run 21: apogee = 4998.5 m
Run 22: apogee = 5179.6 m
Run 23: apogee = 5187.7 m
Run 24: apogee = 5079.6 m
Run 25: apogee = 5202.1 m
Run 26: apogee = 5202.6 m
Run 27: apogee = 5206.2 m
Run 28: apogee = 5196.8 m
Run 29: apogee = 5139.6 m
Run 30: apogee = 5149.3 m
Run 31: apogee = 5221.2 m
Run 32: apogee = 5216.0 m
Run 33: apogee = 5037.5 m
Run 34: apogee = 5139.4 m
Run 35: apogee = 5171.2 m
Run 36: apogee = 5152.9 m
